# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageFont
import random
import pickle as pkl

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.builders import model_builder
from google.protobuf import text_format
import tensorflow as tf
import xml.etree.ElementTree as ET
import time



if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


/home/wc-gpu/MasterThesis/models/research/object_detection/utils/visualization_utils.py:26: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/home/wc-gpu/MasterThesis/models/research/object_detection/utils/visualization_utils.py:26: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wc-gpu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/wc-gpu/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'prod_faster_rcnn_resnet50_coco'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph_new.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [5]:
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
# tar_file = tarfile.open(MODEL_FILE)
# for file in tar_file.getmembers():
#   file_name = os.path.basename(file.name)
#   if 'frozen_inference_graph.pb' in file_name:
#     tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [8]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [9]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.1

In [10]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
# PATH_TO_TEST_IMAGES_DIR = '/Users/mbaltac/Home/MasterThesis/MasterThesis/Kickoff_MihaiBaltac/session_122923'
# TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, f) for f in os.listdir(PATH_TO_TEST_IMAGES_DIR) if f.endswith(".jpg") ]
# SESSIONS_PATH = "/Users/mbaltac/Home/MasterThesis/MasterThesis/labeled_images_shopping_sessions/labeled"
SESSIONS_PATH = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000"

# Size, in inches, of the output images.
IMAGE_SIZE = (30, 20)

In [12]:
def get_labels(fpath: str) :
    try:
        fpath = os.path.abspath(fpath)
        tree = ET.parse(fpath)
        root = tree.getroot()

        img_path = os.path.abspath(os.path.join(os.path.dirname(fpath), "..", root.find("filename").text))
        img_label = []
        size = root.find("size")
        width = float(size.find("width").text)
        height = float(size.find("height").text)
        for child in root.findall("object"):
            name = child.find("name").text
            if name == "add" or name == "remove":
                bndbox = child.find("bndbox")
                xmin = float(bndbox.find("xmin").text)/width
                ymin = float(bndbox.find("ymin").text)/height
                xmax = float(bndbox.find("xmax").text)/width
                ymax = float(bndbox.find("ymax").text)/height

                img_label.append((ymin, xmin, ymax, xmax, name))

        return img_label
    except Exception as ex:
        print (ex)
        return []

In [14]:
bb_intersection_over_union((0,0,1,1), (0,0,0,0))

0.0

In [15]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session(config=tf_config) as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
            
  return output_dict

In [16]:
def filter_detections(output_dict, detection_labels, max_iou=.10, max_filtered=20, conf_tesh=.5, rand_select=2):
    filtered_output_dict = dict()
    
    filtered_idx = []
    detections = output_dict["detection_boxes"]
    for i in range(len(detections)):
        if(output_dict['detection_scores'][i] >= conf_tesh):
            detection = detections[i]
            add_to_filter = True
            for label in detection_labels:
                if bb_intersection_over_union(detection, label) > max_iou:
                    add_to_filter = False
                    break

            for fi in filtered_idx:
                if bb_intersection_over_union(detection, detections[fi]) > .5:
                    add_to_filter = False
                    break

            if add_to_filter:
                filtered_idx.append(i)

            if len(filtered_idx) > max_filtered:
                break
    if len(filtered_idx) > rand_select:
        filtered_idx = random.sample(filtered_idx, rand_select)
    
    filtered_output_dict['detection_boxes'] = output_dict['detection_boxes'][filtered_idx]
    filtered_output_dict['detection_classes'] = output_dict['detection_classes'][filtered_idx]
    filtered_output_dict['detection_scores'] = output_dict['detection_scores'][filtered_idx]
    
    
    return filtered_output_dict

In [ ]:
total_images = 0
sessions = os.listdir(SESSIONS_PATH)
actual_sessions = []

for session in sorted(sessions, reverse=True):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        
        if os.path.exists(os.path.join(session_path, "annotations")):
            actual_sessions.append(session)
            images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
            total_images += len(images)
    except:
        print (session)
        continue

progress = 0

processed = 0
time_avg = 0


for session in sorted(actual_sessions):
    try:
        session_path = os.path.join(SESSIONS_PATH, session)

        for image_fname in sorted(os.listdir(session_path)):
            try:
                boses_path = os.path.join(session_path, "boxes")
                box_path = os.path.join(boses_path, image_fname+".box.better.not_filtered")
                if not os.path.exists(box_path):
                    t0 = time.time()
                    if not image_fname.endswith(".jpg"):
                        continue
                    image = Image.open(os.path.join(session_path, image_fname))
                    # the array based representation of the image will be used later in order to prepare the
                    # result image with boxes and labels on it.
                    image_np = load_image_into_numpy_array(image)
                    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                    label_path = os.path.join(session_path, "annotations", os.path.splitext(image_fname)[0] + ".xml")
                    labels = get_labels(label_path) if os.path.exists(label_path) else []
                    output_dict = run_inference_for_single_image(image_np, detection_graph)
                    output_dict = filter_detections(output_dict, labels)
                    keep = np.where(output_dict["detection_scores"] > 0)[0]
                    output_dict["detection_scores"] = output_dict["detection_scores"][keep]
                    output_dict["detection_classes"] = output_dict["detection_classes"][keep]
                    output_dict["detection_boxes"] = output_dict["detection_boxes"][keep]
#                     detections = output_dict['detection_boxes']
                    
                    
                    # Visualization of the results of a detection.
            #             vis_util.visualize_boxes_and_labels_on_image_array(
            #               image_np,
            #               output_dict['detection_boxes'],
            #               output_dict['detection_classes'],
            #               output_dict['detection_scores'],
            #               category_index,
            #               instance_masks=output_dict.get('detection_masks'),
            #               use_normalized_coordinates=True,
            #               line_thickness=2,
            #               min_score_thresh=.3)

                    if not os.path.exists(boses_path):
                        os.makedirs(boses_path)

                    with open(os.path.join(boses_path, image_fname+".box.better.not_filtered"), "wb") as f:
                        pkl.dump([output_dict["detection_boxes"], output_dict["detection_scores"], labels], f)
                    total_time = time_avg * processed
                    processed += 1

                    t1 = time.time()

                    total = t1-t0
                    print ("total time", total)

                    time_avg = (total_time + total) / processed

                    print ("progress", session, "  : ", progress / total_images, progress, total_images)
                    print ("time", session, "  passed: ", time_avg*processed / 3600 ," h  expected left: ", time_avg* (total_images - progress) / 3600, " h")

                progress += 1
        #         fig,ax = plt.subplots(1, figsize=IMAGE_SIZE)

        #         ax.imshow(image_np)
        #         print("shape", image_np.shape)
        #         for label in labels:
        #             rect = patches.Rectangle(
        #                 (label[1]*image_np.shape[1],label[0]*image_np.shape[0]),
        #                 (label[3]-label[1])*image_np.shape[1],
        #                 (label[2]-label[0])*image_np.shape[0],
        #                 linewidth=5,edgecolor='r',facecolor='none')
        #             ax.add_patch(rect)

        #         for label in detections:
        #             rect = patches.Rectangle(
        #                 (label[1]*image_np.shape[1],label[0]*image_np.shape[0]),
        #                 (label[3]-label[1])*image_np.shape[1],
        #                 (label[2]-label[0])*image_np.shape[0],
        #                 linewidth=5,edgecolor='g',facecolor='none')
        #             ax.add_patch(rect)

        #         plt.show()
            except Exception as ex:
                print (session, ex)
                continue
    except Exception as ex:
        print (session, ex)
        continue

old_ds
max_breaks.pkl
dataset_prod_new_model_multiclass.pkl
dataset_prod_new_model_binary.pkl
total time 3.200087308883667
progress 117653   :  0.0 0 25325
time 117653   passed:  0.0008889131413565742  h  expected left:  22.51172530485524  h
total time 2.2251527309417725
progress 117653   :  3.948667324777888e-05 1 25325
time 117653   passed:  0.001507011122173733  h  expected left:  19.08177482896381  h
total time 2.1998212337493896
progress 117653   :  7.897334649555775e-05 2 25325
time 117653   passed:  0.002118072575993008  h  expected left:  17.87865061395698  h
total time 2.191035032272339
progress 117653   :  0.00011846001974333662 3 25325
time 117653   passed:  0.00272669341829088  h  expected left:  17.261332684490416  h
total time 2.2186176776885986
progress 117653   :  0.0001579466929911155 4 25325
time 117653   passed:  0.003342976106537713  h  expected left:  16.929499598728288  h
total time 2.191862106323242
progress 117653   :  0.00019743336623889436 5 25325
time 117653 

total time 2.2158262729644775
progress 117684   :  0.0019348469891411649 49 25325
time 117684   passed:  0.030912975337770234  h  expected left:  15.627127292749607  h
total time 2.189786672592163
progress 117684   :  0.0019743336623889436 50 25325
time 117684   passed:  0.03152124941349028  h  expected left:  15.621560371097386  h
total time 2.3213002681732178
progress 117684   :  0.0020138203356367228 51 25325
time 117684   passed:  0.032166055043538394  h  expected left:  15.633939907122873  h
total time 2.20753812789917
progress 117684   :  0.0020533070088845015 52 25325
time 117684   passed:  0.032779260079065946  h  expected left:  15.630759244872332  h
total time 2.3697104454040527
progress 117684   :  0.0020927936821322803 53 25325
time 117684   passed:  0.03343751298056707  h  expected left:  15.648756074905387  h
total time 2.2654130458831787
progress 117684   :  0.0021322803553800594 54 25325
time 117684   passed:  0.03406679438220128  h  expected left:  15.652762924229249  

total time 2.1680104732513428
progress 117684   :  0.0039091806515301085 99 25325
time 117684   passed:  0.061599418918291717  h  expected left:  15.53906941632827  h
total time 2.204455614089966
progress 117684   :  0.003948667324777887 100 25325
time 117684   passed:  0.06221176769998338  h  expected left:  15.537542972594858  h
total time 2.202899217605591
progress 117684   :  0.003988153998025666 101 25325
time 117684   passed:  0.06282368414931826  h  expected left:  15.535927539043174  h
total time 2.186476230621338
progress 117684   :  0.0040276406712734455 102 25325
time 117684   passed:  0.06343103865782419  h  expected left:  15.533214447245628  h
total time 2.158273220062256
progress 117684   :  0.004067127344521224 103 25325
time 117684   passed:  0.06403055899673037  h  expected left:  15.528641913610898  h
total time 2.2544820308685303
progress 117684   :  0.004106614017769003 104 25325
time 117684   passed:  0.06465680400530496  h  expected left:  15.530564322074252  h
t

total time 2.2029645442962646
progress 117687   :  0.0058835143139190525 149 25325
time 117687   passed:  0.09214542733298406  h  expected left:  15.465688523568046  h
total time 2.1714773178100586
progress 117687   :  0.005923000987166831 150 25325
time 117687   passed:  0.09274861547682019  h  expected left:  15.463221156483103  h
total time 2.1759252548217773
progress 117687   :  0.00596248766041461 151 25325
time 117687   passed:  0.09335303915871511  h  expected left:  15.460982945930883  h
total time 2.177698850631714
progress 117687   :  0.006001974333662389 152 25325
time 117687   passed:  0.09395795550611281  h  expected left:  15.45884715003515  h
total time 2.1958706378936768
progress 117687   :  0.0060414610069101675 153 25325
time 117687   passed:  0.09456791957219438  h  expected left:  15.457556308255045  h
total time 2.2012624740600586
progress 117687   :  0.006080947680157947 154 25325
time 117687   passed:  0.09517938137054438  h  expected left:  15.456517474051436  h

total time 2.1838672161102295
progress 118859   :  0.007857847976307996 199 25325
time 118859   passed:  0.1227070490519206  h  expected left:  15.415686572392787  h
total time 2.180184841156006
progress 118859   :  0.007897334649555774 200 25325
time 118859   passed:  0.1233126559522417  h  expected left:  15.414081994030212  h
total time 2.1942286491394043
progress 118859   :  0.007936821322803553 201 25325
time 118859   passed:  0.12392216391033596  h  expected left:  15.412972505362777  h
total time 2.2166404724121094
progress 118859   :  0.007976307996051332 202 25325
time 118859   passed:  0.1245378973748949  h  expected left:  15.41263840270682  h
total time 2.1844518184661865
progress 118859   :  0.008015794669299112 203 25325
time 118859   passed:  0.12514468954669106  h  expected left:  15.41120044505869  h
total time 2.250986337661743
progress 118859   :  0.008055281342546891 204 25325
time 118859   passed:  0.12576996352937486  h  expected left:  15.412035384494759  h
total

...

total time 2.191288948059082
progress 123390   :  0.10460019743336624 2649 25325
time 123390   passed:  1.6237244466278329  h  expected left:  13.894179453484051  h
total time 2.194385051727295
progress 123390   :  0.10463968410661402 2650 25325
time 123390   passed:  1.6243339980310902  h  expected left:  13.893539571993578  h
total time 2.1830050945281982
progress 123390   :  0.1046791707798618 2651 25325
time 123390   passed:  1.6249403883351259  h  expected left:  13.892872686693304  h
total time 2.1991984844207764
progress 123392   :  0.10471865745310957 2652 25325
time 123392   passed:  1.6255512768030203  h  expected left:  13.892244289089666  h
total time 2.2342276573181152
progress 123392   :  0.10475814412635735 2653 25325
time 123392   passed:  1.6261718955967197  h  expected left:  13.891699026740326  h
total time 2.191720962524414
progress 123392   :  0.10479763079960513 2654 25325
time 123392   passed:  1.6267807069751987  h  expected left:  13.891052884306866  h
total ti

total time 2.174206018447876
progress 123403   :  0.10657453109575518 2699 25325
time 123403   passed:  1.6543182733986135  h  expected left:  13.863187131080382  h
total time 2.2316462993621826
progress 123403   :  0.10661401776900296 2700 25325
time 123403   passed:  1.6549381751484364  h  expected left:  13.862634658546233  h
total time 2.1802618503570557
progress 123403   :  0.10665350444225075 2701 25325
time 123403   passed:  1.6555438034402021  h  expected left:  13.861962623623663  h
total time 2.182478904724121
progress 123403   :  0.10669299111549851 2702 25325
time 123403   passed:  1.6561500475804034  h  expected left:  13.861295792235097  h
total time 2.2005796432495117
progress 123403   :  0.1067324777887463 2703 25325
time 123403   passed:  1.656761319703528  h  expected left:  13.860671070389502  h
total time 2.1996958255767822
progress 123403   :  0.10677196446199408 2704 25325
time 123403   passed:  1.6573723463217438  h  expected left:  13.860044305413739  h
total ti

total time 2.1805319786071777
progress 123407   :  0.10854886475814413 2749 25325
time 123407   passed:  1.6848278105258965  h  expected left:  13.831517327430051  h
total time 2.1567540168762207
progress 123407   :  0.10858835143139191 2750 25325
time 123407   passed:  1.6854269088639176  h  expected left:  13.830793336097033  h
total time 2.170384168624878
progress 123407   :  0.10862783810463969 2751 25325
time 123407   passed:  1.6860297933552022  h  expected left:  13.830100492441982  h
total time 2.1882448196411133
progress 123407   :  0.10866732477788746 2752 25325
time 123407   passed:  1.686637639138436  h  expected left:  13.82944839385104  h
total time 2.194413185119629
progress 123407   :  0.10870681145113524 2753 25325
time 123407   passed:  1.6872471983565247  h  expected left:  13.828810370843671  h
total time 2.174086093902588
progress 123407   :  0.10874629812438302 2754 25325
time 123407   passed:  1.6878511111603867  h  expected left:  13.828126108893317  h
total tim

total time 2.2018308639526367
progress 123409   :  0.11052319842053307 2799 25325
time 123409   passed:  1.7153652567333666  h  expected left:  13.800113490419935  h
total time 2.1954002380371094
progress 123409   :  0.11056268509378085 2800 25325
time 123409   passed:  1.7159750901328212  h  expected left:  13.799478366741091  h
total time 2.203400135040283
progress 123409   :  0.11060217176702863 2801 25325
time 123409   passed:  1.7165871457258879  h  expected left:  13.798861124314739  h
total time 2.2190864086151123
progress 123409   :  0.1106416584402764 2802 25325
time 123409   passed:  1.7172035586171697  h  expected left:  13.79827889787175  h
total time 2.175097942352295
progress 123409   :  0.11068114511352418 2803 25325
time 123409   passed:  1.7178077524900455  h  expected left:  13.797598502703565  h
total time 2.1864705085754395
progress 123409   :  0.11072063178677197 2804 25325
time 123409   passed:  1.7184151054090944  h  expected left:  13.796943525461039  h
total ti

total time 2.179502487182617
progress 123416   :  0.11249753208292201 2849 25325
time 123416   passed:  1.7459659287664637  h  expected left:  13.76923867191405  h
total time 2.1783883571624756
progress 123416   :  0.1125370187561698 2850 25325
time 123416   passed:  1.7465710366434535  h  expected left:  13.768566835693305  h
total time 2.2094500064849854
progress 123416   :  0.11257650542941758 2851 25325
time 123416   passed:  1.7471847727563659  h  expected left:  13.767963037491784  h
total time 2.2075235843658447
progress 123416   :  0.11261599210266535 2852 25325
time 123416   passed:  1.747797973752023  h  expected left:  13.767355017220194  h
total time 2.22780179977417
progress 123416   :  0.11265547877591313 2853 25325
time 123416   passed:  1.7484168075852937  h  expected left:  13.766791345499902  h
total time 2.1716856956481934
progress 123416   :  0.11269496544916091 2854 25325
time 123416   passed:  1.7490200536118625  h  expected left:  13.766104947359777  h
total time